In [6]:
import pennylane as qml
from qiskit.quantum_info import random_unitary
from qiskit.circuit.library import UnitaryGate
from sync import cost as cost_func
from sync import circuit, constant
import pennylane.numpy as np
import pennylane as qml
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def target_func(num_qubits, num_layers, active_blocks):
	num_blocks = num_qubits*(num_qubits-1)//2
	thetas = np.random.uniform(0, 2*np.pi, constant.num_params_on_active_blocks(active_blocks), requires_grad=True)

	def u3cry(thetas):
		k, index_params = 0, 0
		for _ in range(num_layers):
			for i in range(num_qubits):
				for j in range(i + 1, num_qubits):
					if active_blocks[k] == 1:
						qml.U3(thetas[index_params],thetas[index_params + 1],thetas[index_params + 2],i)
						qml.U3(thetas[index_params + 3],thetas[index_params + 4],thetas[index_params + 5],j)
						qml.CNOT([j, i])
						index_params += 6
					k += 1

	target = UnitaryGate(random_unitary(2**num_qubits))
	def cost(thetas):
		matrix = qml.matrix(u3cry, wire_order=list(range(num_qubits)))(thetas)
		return cost_func.c_hst(matrix, target)

	steps = 1000
	costs = []
	opt = qml.AdamOptimizer(stepsize = 0.01)
	for n in range(steps):
		thetas, prev_cost = opt.step_and_cost(cost, thetas)
		costs.append(prev_cost)
		if prev_cost < constant.error:
			print("Achieved error threshold at step", n)
			break
		
	return costs[-1]

In [7]:
num_qubits = 3
num_layers = 8
active_blocks = np.ones((num_qubits*(num_qubits-1)//2) * num_layers) # num_blocks = num_qubits*(num_qubits-1)//2
def check(active_blocks):
    cost = target_func(num_qubits, num_layers, active_blocks)
    print(cost)
    if cost < constant.error:
        return True
    return False
    # return xs[2] == 0 and xs[5] == 1
for i in range(len(active_blocks)):
    previous_sum = np.sum(active_blocks)
    for j in range(len(active_blocks)):
        if active_blocks[j] == 1:
            active_blocks[j] = 0
            print("I'm testing ...")
            k = check(active_blocks)
            print(f"{active_blocks} is {k}")
            if k == False:
                active_blocks[j] = 1
    if np.sum(active_blocks) == previous_sum:
        break
            
print(active_blocks)

I'm testing ...
Achieved error threshold at step 199
9.928432964656508e-09
[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] is True
I'm testing ...


KeyboardInterrupt: 